In [ ]:
import os
import cv2
import time
import imutils
import numpy as np
import mediapipe as mp
import tensorflow as tf
from tensorflow import keras
from collections import Counter
from matplotlib import pyplot as plt
from PIL import ImageFont, ImageDraw, Image

In [ ]:
mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils
handsModule = mp.solutions.hands
drawingModule = mp.solutions.drawing_utils
mp_model = handsModule.Hands(static_image_mode = True)

def mediapipe_detection(image, model):
    
    results = mp_model.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    #image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    #image.flags.writeable = False                  # Image is no longer writeable
    #results = model.process(image)                 # Make prediction
    #image.flags.writeable = True                   # Image is now writeable 
    #image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results

def draw_landmarks(image, results):
    
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
    
def draw_styled_landmarks(image, results):

    if results.multi_hand_landmarks != None:
        for handLandmarks in results.multi_hand_landmarks:
            drawingModule.draw_landmarks(image, handLandmarks, handsModule.HAND_CONNECTIONS, 
                                        drawingModule.DrawingSpec(color=(121, 22, 76), thickness=2, circle_radius=4), 
                                        drawingModule.DrawingSpec(color=(0, 255, 0), thickness=2, circle_radius=2))
    
def extract_keypoints(results):

    kp_list = []

    if results.multi_hand_landmarks != None:
        for data_point in results.multi_hand_landmarks[0].landmark:
            kp_list.append([data_point.x, data_point.y, data_point.z])

        keypoints = np.array(kp_list)
        temp = keypoints.flatten()
    
        return temp

In [ ]:
model = keras.models.load_model('model.h5')

actions = np.array(['a', 'b', 'backspace', 'c', 'ç', 'd', 'e', 'ə', 'enter', 'f', 'g', 'ğ', 'h', 'ı', 'i', 'j', 'k', 'l', 'm', 
                    'n', 'o', 'ö', 'p', 'q', 'r', 's', 'ş', ' ', 't', 'u', 'ü', 'v', 'x', 'y', 'z'])

label_map = {label:num for num, label in enumerate(actions)}

In [ ]:
file = open("lexicon.txt", encoding = "utf8")
text = file.read()

text = text.replace('\n', ' ')
words = text.split(sep = ' ')
words = [word.lower() for word in words]

stop = ['da', 'də', 'nə']

for i in range(0, len(words)):
    if words[i].isalpha() != True:
        words[i] = ''

for i in range(0, len(words)):
    if len(words[i]) == 1:
        words[i] = ''

for i in range(0, len(words)):
    for item in stop:
        if words[i] == item:
            words[i] = ''

words[:] = [x for x in words if x != '']

c = Counter(words)
most_words = c.most_common(10)
less_words = c.most_common(len(words))[-10:]

unique_words = []
for i in words:
    if i not in unique_words:
        unique_words.append(i)

alphabet = ["a", 'b', 'c', 'ç', 'd', 'e', 'ə', 'f', 'g', 'ğ', 'h', 'x', 'ı', 'i', 'j', 'k',
            'q', 'l', 'm', 'n', 'o', 'ö', 'p', 'r', 's', 'ş', 't', 'u', 'ü', 'v', 'y', 'z']

keys = []
for letter in alphabet:
    keys.append(letter + '| ')
for letter1 in alphabet:
    for letter2 in alphabet:
        keys.append(letter1 + '|' + letter2)
for letter in alphabet:
    keys.append(' |' + letter)

probas = dict.fromkeys(keys)
probas = {x: 0 for x in probas}

for i in range(0, len(unique_words)):
    for j in range(0, len(unique_words[i]) + 1):
        if j == 0:
            probas[unique_words[i][j] + '| '] = probas[unique_words[i][j] + '| '] + 1
        if j == len(unique_words[i]):
            probas[' |' + unique_words[i][j - 1]] = probas[' |' + unique_words[i][j - 1]] + 1
        else:
            probas[unique_words[i][j] + '|' + unique_words[i][j - 1]] = probas[unique_words[i][j] + '|' + unique_words[i][j - 1]] + 1

freqs = probas.copy()

given = []
for i in range(0, len(unique_words)):
    for j in range(0, len(unique_words[i])-1):
        if j == 0:
            given.append(' ')
        given.append(unique_words[i][j])

temp = {}
for item in given:
    if item in temp:
        temp[item] += 1
    else:
        temp[item] = 1

keys = []
keys.append(' ')
for letter in alphabet:
    keys.append(letter)
letters = dict.fromkeys(keys)
        
for i in letters:
    for j in temp:
        if i == j:
            letters[i] = temp[j]

for i in probas:
    probas[i] = probas[i] / letters[i[-1]]

freqs[' | '] = 0
probas[' | '] = 0.0

In [ ]:
def Model_Predict(temp, width):
    
    hand_pred = model.predict(temp.reshape(1, -1))
    temp = hand_pred[0].copy()
    temp.sort()
    res = []
    proba = []
    
    for i in range(1, width + 1):
        idx = np.where(hand_pred[0] == temp[-i])[0][0]
        proba.append(temp[-i])
        res.append(list(label_map.keys())[idx])
        
    res = np.array(res)
    proba = np.array(proba)
    return proba, res

In [ ]:
def Beam_Camera(proba_list, res_list, width):
    
    proba_total = proba_list[0]
    res_total = res_list[0]
    proba_total = proba_total.tolist()
    res_total = res_total.tolist()
    
    for i in range(1, len(proba_list)):
        proba, res = proba_list[i], res_list[i]
        outer = np.multiply.outer(proba_total, proba)
        temp = outer.copy()
        temp = temp.flatten()
        temp.sort()
        res_total_temp = res_total.copy()
        
        for j in range(0, width):
            proba_total[j] = temp[-(j+1)]
            idx = np.where(outer == temp[-(j+1)])
            res_total[j] = res_total_temp[idx[0][0]] + res[idx[1][0]]
    
    letter_probas = proba_total.copy()
    
    for i in range(0, len(res_total)):
        for j in range(0, len(res_total[i])):
            if j == 0:
                letter_probas[i] = letter_probas[i] * probas[res_total[i][0] + '| ']
            else:
                letter_probas[i] = letter_probas[i] * probas[res_total[i][j] + '|' + res_total[i][j-1]]
    
    overall_probas = []
    for i in range(0, len(proba_total)):
        overall_probas.append(proba_total[i] * letter_probas[i] * 10000)
    
    return res_total, proba_total, letter_probas, overall_probas

In [ ]:
def Demo():
    width = 3
    flag_box = False
    proba_list, res_list = [], []
    sequence = []
    sentence = [' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ']
    
    cap = cv2.VideoCapture(0)
    
    with handsModule.Hands(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
        while cap.isOpened():
            
            key = cv2.waitKey(10) & 0xFF
            ret, frame = cap.read()
            frame = imutils.resize(frame, width=1920)
            image, results = mediapipe_detection(frame, holistic)
            draw_styled_landmarks(image, results)
            image = cv2.flip(image, 1)
            keypoints = extract_keypoints(results)
            cv2.rectangle(image, (0, 0), (540, 40), (245, 117, 16), -1)
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            pil_image = Image.fromarray(image)
            font = ImageFont.truetype("C:\Windows\Fonts\\arial.ttf", 35)
            draw = ImageDraw.Draw(pil_image)
            
            if keypoints is None :
                keypoints = np.zeros((63, ))
            
            sequence.append(keypoints)
            sequence = sequence[-30:]
            
            if len(sequence) == 30:
                sequence = sequence[-20:]
                my_array = np.array(sequence)
                sequence = []
                
                proba, res = Model_Predict(my_array, width)
                print(res[0])
                
                if res[0] != ' ' and res[0] != 'enter' and res[0] != 'backspace' and flag_box == False:
                    proba_list.append(proba)
                    res_list.append(res)
                    sentence.append(res[0])
                
                if res[0] == 'backspace':
                    sentence = [' '] + sentence[:len(sentence) - 1]
                    proba_list = proba_list[:-1]
                    res_list = res_list[:-1]
                    if flag_box == True:
                        proba_list, res_list = [], []
                        sentence = [' '] * 10
                        flag_box = False
                
                if res[0] == 'enter' and len(res_list) != 0:
                    res_total, proba_total, letter_probas, overall_probas = Beam_Camera(proba_list, res_list, width)
                    final_res = [x for _, x in sorted(zip(overall_probas, res_total), reverse = True)]
                    final_probas = proba_total.copy()
                    final_probas.sort(reverse = True)
                    proba_list, res_list = [], []
                    sentence = []
                    probability = []
                    print(res_total)
                    print(proba_total)
                    print(letter_probas)
                    flag_box = True
                
                if len(sentence) > 10:
                    sentence = sentence[-10:]
            
            if key == ord('d'):
                    proba_list, res_list = [], []
                    sentence = [' '] * 10
                    flag_box = False
            
            draw.text((10, 0), ' '.join(sentence), font = font)
            
            if flag_box == True:
                draw.rectangle([(540, 0), (1080, 120)], fill ="#ff0000")
                draw.text((600, 0), final_res[0] + ': ' + str(round(final_probas[0] * 100, 1)) + '%', font = font)
                draw.text((600, 40), final_res[1] + ': ' + str(round(final_probas[1] * 100, 1)) + '%', font = font)
                draw.text((600, 80), final_res[2] + ': ' + str(round(final_probas[2] * 100, 1)) + '%', font = font)
            
            image = np.asarray(pil_image)
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
            
            cv2.waitKey(10)
            cv2.imshow('OpenCV Feed', image)
            
            if key == ord('q'):
                break
        
        cap.release()
        cv2.destroyAllWindows()

In [ ]:
Demo()